## Tools in Langchain
1. How to create tools <br>
2. How to use built-in tools and toolkit <br>
3. How to use chat models to call tools <br>
4. How to pass tool outputs to chat model <br>

This @tool decoration is the simplest way to define a custom tool. The decorator uses the function name by default, but this can be overriden by passing a string as the first argument. Additionally, the decorator will use the function's docstring as the tool's description- so a docstring must be provided

In [86]:
modelName = "qwen/qwen3-32b"

In [87]:
## @tool decorator
from langchain_core.tools import tool

@tool
def division(a:int, b:int) -> int:
    """This function divides two numbers"""
    return a/b




In [88]:
print(division.name)
print(division.description)
print(division.args)


division
This function divides two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [89]:
### async implementation

@tool
async def multiply(a:int, b:int) -> int:
    """This function multiplies 2 number"""
    return a*b



In [90]:
print(division.name)
print(division.description)
print(division.args)

division
This function divides two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [91]:
from typing import Annotated

@tool
def multiplyTwoNumbers(a: Annotated[int, "First number"], b: Annotated[int, "second number"]) -> int :
    """Multiply the two numbers"""
    return a * b

In [92]:
print(multiplyTwoNumbers.description)
print(multiplyTwoNumbers.args)

Multiply the two numbers
{'a': {'description': 'First number', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'second number', 'title': 'B', 'type': 'integer'}}


### Structured Tool
The StructuredTool.from_function class method provides a bit more configurability then @tool decorator, without requiring much additional code.

In [93]:
from langchain_core.tools import StructuredTool

def multiply(a: int, b: int) -> int :
    """Multiply two numbers"""
    print("Multiply called")
    return a*b

async def asyncMultiply(a:int, b: int) -> int:
    """Multiply two numbers"""
    print("Async multiplied called")
    return a*b

calculator = StructuredTool.from_function(multiply, asyncMultiply)

print(calculator.invoke({"a":2, "b":3}))
print(await calculator.ainvoke({"a":2, "b":3}))


Multiply called
6
Async multiplied called
6


### Inbuilt Tools

https://docs.langchain.com/oss/python/integrations/tools

### WikiPedia integration

In [94]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


apiWrapper = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=500)
tool = WikipediaQueryRun(api_wrapper=apiWrapper)

response = tool.invoke({"query": "langchain"})
print(response)


Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: Vector database
Summary: A vector database, vector store or vector search engine is a database that uses the vector space model to store vectors


### Duckduck go integration

In [95]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchRun()
result = DuckDuckGoSearchResults()

searchResponse = search.invoke("Price of gold in india today")
resultResponse = result.invoke("Price of gold in india today")

print(searchResponse)
print("------")
print(resultResponse)

3 days ago · Today gold price in India for 24 karat gold is 124,741 rupees per 10 grams. Gold price in India for 22 karat gold is 114,346 rupees per 10 grams. Gold rate per tola (24 karat) is ₹145,495.45 - One tola is equal to 180 troy grains which is 11.6638038 grams. This Gold calculator uses the current Gold Rate Today Calculator rate today for India only. 21 hours ago · The price of gold in India today is ₹12,704 per gram for 24 karat gold, ₹11,645 per gram for 22 karat gold and ₹9,528 per gram for 18 karat gold (also called 999 gold). The current price of 24-carat gold in India is Rs 124,540 per 10 gms, which is gain of 1.28% from its previous close. Gold And Silver Rates Today in India : On March 28, 2025, the price of gold experienced a flat movement in the morning. Gold and silver prices today In India : Gold hit a ... What Is The Price Of 22kt, 24kt Gold Rates Today In India Across Key Cities On September 09?
------
snippet: The price of 1 gram gold rate today for 24 carat gol

### call tool with my LLM model

In [96]:
from langchain_community.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools import tool
from langchain.chat_models import init_chat_model


wikiApi = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=100)
wikiTool = WikipediaQueryRun(api_wrapper=wikiApi)

searchTool = DuckDuckGoSearchRun()

@tool
def multiply(a:int, b:int) -> int :
    """Mutliple two numbers"""
    return a*b

tools = [wikiTool, searchTool, multiply]
llm = init_chat_model(modelName)
llm_with_tools = llm.bind_tools(tools=tools)

multiplyQuery = "What is 2*3"
addQuery = "What is 2+3"
todayNews = "tell me top 5 news"

multiplyResponse = llm_with_tools.invoke(multiplyQuery)
addResponse = llm_with_tools.invoke(addQuery)
newsResponse = llm_with_tools.invoke(todayNews)

print("--------")
print(multiplyResponse)
print(multiplyResponse.tool_calls)

print("--------")

print(addResponse)
print(addResponse.tool_calls)

print("--------")

print(newsResponse)
print(newsResponse.tool_calls)

toolNamesMap = {"wikiPedia": wikiTool,"multiply": multiply, "duckduckgo_search": searchTool}
messages = []
## Executing the tool calls
for tool in multiplyResponse.tool_calls:
    selected_tool = toolNamesMap[tool["name"].lower()]
    tool_msg = selected_tool.invoke(tool)
    messages.append(tool_msg)

print("--------")
print(messages)

## lets modifying these for all responses
allResponses = [newsResponse, addResponse, multiplyResponse]
allReponseMesages = []
for res in allResponses:
    print("Res",res)
    for tool in res.tool_calls:
        selected_tool = toolNamesMap[tool["name"].lower()]
        tool_msg = selected_tool.invoke(tool)
        allReponseMesages.append(tool_msg)

print("--------")
print(allReponseMesages)










ValueError: Unable to infer model provider for model='qwen/qwen3-32b', please specify model_provider directly.

In [ ]:
from langchain_core.messages import HumanMessage


allMessages=[HumanMessage(multiplyQuery)]


for tool in multiplyResponse.tool_calls:
    selected_tool = toolNamesMap[tool["name"].lower()]
    tool_msg = selected_tool.invoke(tool)
    allMessages.append(tool_msg)

print(allMessages)
finalResponse = llm_with_tools.invoke(allMessages)
print(finalResponse)


[HumanMessage(content='What is 2*3', additional_kwargs={}, response_metadata={}), ToolMessage(content='6', name='multiply', tool_call_id='e37vfwdqe')]
content='The result of 2*3 is 6.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 48, 'total_tokens': 60, 'completion_time': 0.020089717, 'completion_tokens_details': None, 'prompt_time': 0.002282235, 'prompt_tokens_details': None, 'queue_time': 0.056015245, 'total_time': 0.022371952}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--d80c3f66-0215-4484-b709-8abbc037b46f-0' usage_metadata={'input_tokens': 48, 'output_tokens': 12, 'total_tokens': 60}


In [ ]:
searchTool.name

'duckduckgo_search'

### Another example

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage

query = "What is langchain and what is 2*5"
messages = [HumanMessage(content=query)]

toolNamesMap = {
    "wikipedia": wikiTool,
    "multiply": multiply,
    "duckduckgo_search": searchTool,
}

# ---- 1st call: model decides which tools to call ----
ai_msg = llm_with_tools.invoke(messages)
print("Model returned tool calls:", ai_msg.tool_calls)

messages.append(ai_msg)

# ---- Run the tools and append ToolMessages ----
for tool_call in ai_msg.tool_calls:
    name = tool_call["name"]
    args = tool_call["args"]
    call_id = tool_call["id"]

    selected_tool = toolNamesMap[name.lower()]
    tool_result = selected_tool.invoke(args)  # note: pass args, not the whole tool_call

    # Normalize to string content
    if hasattr(tool_result, "content"):
        content = tool_result.content
    else:
        content = str(tool_result)

    # IMPORTANT: only content + tool_call_id are required
    messages.append(ToolMessage(content, tool_call_id=call_id))

# ---- 2nd call: model should now answer using tool results ----
final_response = llm_with_tools.invoke(messages)
print("Final content:", final_response.content)
print("Final tool_calls:", getattr(final_response, "tool_calls", None))


Model returned tool calls: [{'name': 'wikipedia', 'args': {'query': 'langchain'}, 'id': 'cc9r5kaps', 'type': 'tool_call'}, {'name': 'multiply', 'args': {'a': 2, 'b': 5}, 'id': '8exezeqar', 'type': 'tool_call'}]
Final content: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into various applications, such as chatbots, virtual assistants, and more. It provides a set of tools and APIs that make it easier to work with LLMs, including tasks like text generation, question-answering, and text classification.

As for the math problem, 2*5 equals 10.
Final tool_calls: []
